# **Search and Download Atmospheric-corrected Landsat 1-9 imagery from Google Earth Engine**

The codes are tested inside Google Colab environment using Hong Kong as the study area. Results are exported to Google Drive. Refer to the repository for details. https://github.com/ivanhykwong/Habitat-Type-Quality-HK

# Step 1 - Set up Py6S in Google Colab
(For atmospheric correction of Landsat 1-5 MSS data)

(No need to run this part for Landsat 4-9 data since Level-2 products are already atmospheric corrected)

In [ ]:
!gfortran -v
!wget http://rtwilson.com/downloads/6SV-1.1.tar
!tar xvf 6SV-1.1.tar
!cd 6SV1.1

**Manual work required before executing the subsequent code**

Refer to comments below

In [ ]:
# modify "makefile" from "FC = g77 $(FFLAGS)" to "FC = gfortran -std=legacy -ffixed-line-length-none -ffpe-summary=none $(FFLAGS)"
# upload modified "makefile" to /content/6SV1.1

import os
os.chdir("/content/6SV1.1")
!ls
!make
os.environ["PATH"]="/content/6SV1.1:"+os.environ["PATH"]
# test
!sixsV1.1 < /content/Examples/Example_In_1.txt
!pip install Py6S
from Py6S import *
SixS.test()

**Functions created by Sam Murphy**

Modified from https://github.com/samsammurphy/gee-atmcorr-S2

In [ ]:
"""
atmospheric.py, Sam Murphy (2016-10-26)

Atmospheric water vapour, ozone and AOT from GEE

Usage
H2O = Atmospheric.water(geom,date)
O3 = Atmospheric.ozone(geom,date)
AOT = Atmospheric.aerosol(geom,date)

"""


import ee

class Atmospheric():

  def round_date(date,xhour):
    """
    rounds a date of to the closest 'x' hours
    """
    y = date.get('year')
    m = date.get('month')
    d = date.get('day')
    H = date.get('hour')
    HH = H.divide(xhour).round().multiply(xhour)
    return date.fromYMD(y,m,d).advance(HH,'hour')

  def round_month(date):
    """
    round date to closest month
    """
    # start of THIS month
    m1 = date.fromYMD(date.get('year'),date.get('month'),ee.Number(1))

    # start of NEXT month
    m2 = m1.advance(1,'month')

    # difference from date
    d1 = ee.Number(date.difference(m1,'day')).abs()
    d2 = ee.Number(date.difference(m2,'day')).abs()

    # return closest start of month
    return ee.Date(ee.Algorithms.If(d2.gt(d1),m1,m2))



  def water(geom,date):
    """
    Water vapour column above target at time of image aquisition.

    (Kalnay et al., 1996, The NCEP/NCAR 40-Year Reanalysis Project. Bull.
    Amer. Meteor. Soc., 77, 437-471)
    """

    # Point geometry required
    centroid = geom.centroid()

    # H2O datetime is in 6 hour intervals
    H2O_date = Atmospheric.round_date(date,6)

    # filtered water collection
    water_ic = ee.ImageCollection('NCEP_RE/surface_wv').filterDate(H2O_date, H2O_date.advance(1,'month'))

    # water image
    water_img = ee.Image(water_ic.first())

    # water_vapour at target
    water = water_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('pr_wtr')

    # convert to Py6S units (Google = kg/m^2, Py6S = g/cm^2)
    water_Py6S_units = ee.Number(water).divide(10)

    return water_Py6S_units



  def ozone(geom,date):
    """
    returns ozone measurement from merged TOMS/OMI dataset

    OR

    uses our fill value (which is mean value for that latlon and day-of-year)

    """

    # Point geometry required
    centroid = geom.centroid()

    def ozone_measurement(centroid,O3_date):

      # filtered ozone collection
      ozone_ic = ee.ImageCollection('TOMS/MERGED').filterDate(O3_date, O3_date.advance(1,'month'))

      # ozone image
      ozone_img = ee.Image(ozone_ic.first())

      # ozone value IF TOMS/OMI image exists ELSE use fill value
      ozone = ee.Algorithms.If(ozone_img,\
      ozone_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone'),\
      ozone_fill(centroid,O3_date))

      return ozone

    def ozone_fill(centroid,O3_date):
      """
      Gets our ozone fill value (i.e. mean value for that doy and latlon)

      you can see it
      1) compared to LEDAPS: https://code.earthengine.google.com/8e62a5a66e4920e701813e43c0ecb83e
      2) as a video: https://www.youtube.com/watch?v=rgqwvMRVguI&feature=youtu.be

      """

      # ozone fills (i.e. one band per doy)
      ozone_fills = ee.ImageCollection('users/samsammurphy/public/ozone_fill').toList(366)

      # day of year index
      jan01 = ee.Date.fromYMD(O3_date.get('year'),1,1)
      doy_index = date.difference(jan01,'day').toInt()# (NB. index is one less than doy, so no need to +1)

      # day of year image
      fill_image = ee.Image(ozone_fills.get(doy_index))

      # return scalar fill value
      return fill_image.reduceRegion(reducer=ee.Reducer.mean(), geometry=centroid).get('ozone')

    # O3 datetime in 24 hour intervals
    O3_date = Atmospheric.round_date(date,24)

    # TOMS temporal gap
    TOMS_gap = ee.DateRange('1994-11-01','1996-08-01')

    # avoid TOMS gap entirely
    ozone = ee.Algorithms.If(TOMS_gap.contains(O3_date),ozone_fill(centroid,O3_date),ozone_measurement(centroid,O3_date))

    # fix other data gaps (e.g. spatial, missing images, etc..)
    ozone = ee.Algorithms.If(ozone,ozone,ozone_fill(centroid,O3_date))

    #convert to Py6S units
    ozone_Py6S_units = ee.Number(ozone).divide(1000)# (i.e. Dobson units are milli-atm-cm )

    return ozone_Py6S_units


  def aerosol(geom,date):
    """
    Aerosol Optical Thickness.

    try:
      MODIS Aerosol Product (monthly)
    except:
      fill value
    """

    def aerosol_fill(date):
      """
      MODIS AOT fill value for this month (i.e. no data gaps)
      """
      return ee.Image('users/samsammurphy/public/AOT_stack')\
               .select([ee.String('AOT_').cat(date.format('M'))])\
               .rename(['AOT_550'])


    def aerosol_this_month(date):
      """
      MODIS AOT original data product for this month (i.e. some data gaps)
      """
      # image for this month
      img =  ee.Image(\
                      ee.ImageCollection('MODIS/061/MOD08_M3')\
                        .filterDate(Atmospheric.round_month(date))\
                        .first()\
                     )

      # fill missing month (?)
      img = ee.Algorithms.If(img,\
                               # all good
                               img\
                               .select(['Aerosol_Optical_Depth_Land_Mean_Mean_550'])\
                               .divide(1000)\
                               .rename(['AOT_550']),\
                              # missing month
                                aerosol_fill(date))

      return img


    def get_AOT(AOT_band,geom):
      """
      AOT scalar value for target
      """
      return ee.Image(AOT_band).reduceRegion(reducer=ee.Reducer.mean(),\
                                 geometry=geom.centroid())\
                                .get('AOT_550')


    after_modis_start = date.difference(ee.Date('2000-03-01'),'month').gt(0)

    AOT_band = ee.Algorithms.If(after_modis_start, aerosol_this_month(date), aerosol_fill(date))

    AOT = get_AOT(AOT_band,geom)

    AOT = ee.Algorithms.If(AOT,AOT,get_AOT(aerosol_fill(date),geom))
    # i.e. check reduce region worked (else force fill value)

    return AOT

**Py6S function**

In [1]:
# Define Py6S function
# Modified from https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md

def func1(img):
  L1 = ee.Image(img)
  date = L1.date()

  info = L1.getInfo()['properties']
  scene_date = datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
  solar_z = 90 - info['SUN_ELEVATION']

  h2o = Atmospheric.water(geom,date).getInfo()
  o3 = Atmospheric.ozone(geom,date).getInfo()
  aot = Atmospheric.aerosol(geom,date).getInfo()

  SRTM = ee.Image('CGIAR/SRTM90_V4')# Shuttle Radar Topography mission covers *most* of the Earth
  alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
  km = alt/1000 # i.e. Py6S uses units of kilometers

  # Instantiate
  s = SixS()

  # Atmospheric constituents
  s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
  s.aero_profile = AeroProfile.Maritime # https://github.com/robintw/Py6S/blob/master/Py6S/Params/aeroprofile.py [NoAerosols, Continental, Maritime, Urban, Desert, BiomassBurning, Stratospheric]
  s.aot550 = aot

  # Earth-Sun-satellite geometry
  s.geometry = Geometry.User()
  s.geometry.view_z = 0               # always NADIR
  s.geometry.solar_z = solar_z        # solar zenith angle
  s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
  s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
  s.altitudes.set_sensor_satellite_level()
  s.altitudes.set_target_custom_altitude(km)

  def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.LANDSAT_MSS_B1,   # ['B4','B5','B6','B7']  for L1 to L3; ['B1','B2','B3','B4'] for L4 & L5
        'B2':PredefinedWavelengths.LANDSAT_MSS_B2,
        'B3':PredefinedWavelengths.LANDSAT_MSS_B3,
        'B4':PredefinedWavelengths.LANDSAT_MSS_B4
        }
    return Wavelength(bandSelect[bandname])

  def dn_to_rad(bandname):
    radiance = ee.Algorithms.Landsat.calibratedRadiance(L1)
    rad = radiance.select(bandname)
    return rad

  def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    # radiance to surface reflectance
    rad = dn_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    return ref

  # all wavebands
  output = L1.select('QA_PIXEL')
  # bnames = ['B4','B5','B6','B7']  # for L1 to L3
  bnames = ['B1','B2','B3','B4'] # for L4 & L5
  for band in bnames:
    print(band)
    output = output.addBands(surface_reflectance(band))

  output = output.select(bnames)
  return output


# Step 2 - Define function to remove clouds using cloud mask

In [2]:
# cloud mask
# https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/

def bitwiseExtract(input, fromBit, toBit):
  maskSize = ee.Number(1).add(toBit).subtract(fromBit)
  mask = ee.Number(1).leftShift(maskSize).subtract(1)
  return input.rightShift(fromBit).bitwiseAnd(mask)

def cloudmask(img):
  qapixel = img.select('QA_PIXEL')
  Cloud = bitwiseExtract(qapixel, 3, 3).eq(0)
  CloudShadow = bitwiseExtract(qapixel, 4, 4).eq(0)
  ImageMask = Cloud.bitwiseAnd(CloudShadow)
  Masked = img.updateMask(ImageMask)
  return Masked

# Radiometric Saturation Quality Assessment band (QA_RADSAT)
def qaradsat(img):
  qa = img.select('QA_RADSAT')
  ImageMask = qa.eq(0)
  Masked = img.updateMask(ImageMask)
  return Masked


# Step 3 - Initialize session

Import required libraries

In [ ]:
import ee
from datetime import datetime
import math
import os
import sys
import pandas as pd

In [ ]:
# import Py6S library, if step 1 has been correctly done
from Py6S import *

**Initialize Google Earth Engine session**

Need enter verification using GEE account

In [ ]:
ee.Authenticate()
ee.Initialize()

# Step 4 - Execute processing for Landsat 1-5 MSS data

Results are saved as GEE assets

In [ ]:
# Define function to chain the above process

def preprocess_mss(image):
  d = cloudmask(image)
  d = qaradsat(d)
  d = func1(d)
  d1 = d.clip(aoi)

  # export to asset
  fname = ee.String(image.get('system:index')).getInfo()
  export = ee.batch.Export.image.toAsset(
      image = d1,
      description= 'L5MS_' + fname,
      assetId = 'users/xxx/Landsat_mss/' +'L5MS_' + fname, # Manually create image collection in GEE asset first
      region = aoi,
      scale = 30)
  export.start()
  print('exporting ' +fname + '--->done')


In [ ]:
aoi = ee.Geometry.Polygon([[[113.810, 22.580],[113.810, 22.140],[114.460, 22.140],[114.460, 22.580]]])
geom = aoi
df_metadata = pd.DataFrame()

#('LANDSAT/LM01/C02/T1') ('LANDSAT/LM01/C02/T2') LANDSAT/LM02/C02/T1  LANDSAT/LM02/C02/T2  LANDSAT/LM03/C02/T1  LANDSAT/LM03/C02/T2  ['B4','B5','B6','B7']
# LANDSAT/LM04/C02/T1  LANDSAT/LM04/C02/T2  LANDSAT/LM05/C02/T1  LANDSAT/LM05/C02/T2  ['B1','B2','B3','B4']

dataset = (ee.ImageCollection('LANDSAT/LM05/C02/T2')  # change ImageCollection according to above
  .filterBounds(aoi)
  .filterDate('1970-01-01', '2022-12-31')
  .filter(ee.Filter.lte('CLOUD_COVER_LAND', 20)))

col_length = dataset.size().getInfo()
print(col_length)
if col_length > 0:
  col_list = dataset.toList(col_length)
  for i in range(0,col_length):
    print(i)
    img = ee.Image(col_list.get(i))
    p = img.getInfo()['properties']
    df_metadata1 = pd.DataFrame({'index': [p['system:index']], 'GEE_ID': [img.getInfo()['id']], 'LANDSAT_PRODUCT_ID': [p['LANDSAT_PRODUCT_ID']],
                               'SPACECRAFT_ID': [p['SPACECRAFT_ID']], 'SENSOR_ID': [p['SENSOR_ID']],
                               'Date': [p['DATE_ACQUIRED']], 'Time':[p['SCENE_CENTER_TIME']],
                               'WRS_TYPE':[p['WRS_TYPE']], 'WRS_PATH':[p['WRS_PATH']], 'WRS_ROW':[p['WRS_ROW']],
                               'CloudCover':[p['CLOUD_COVER']], 'SunAzimuth': [p['SUN_AZIMUTH']], 'SunElevation': [p['SUN_ELEVATION']], 'SunZenith': [90-p['SUN_ELEVATION']]})
    df_metadata = pd.concat([df_metadata, df_metadata1])
    preprocess_mss(img)

df_metadata.to_csv('df_metadata_mss.csv')
from google.colab import files
files.download('df_metadata_mss.csv')

# Step 5 - Execute processing for Landsat 4-9 TM/ETM+/OLI data

Results are saved as GEE assets

In [ ]:
# Define function to chain the above process

def preprocess(image):
  d = cloudmask(image)
  d = qaradsat(d)
  d = d.select('SR_B.')
  d_scale = d.multiply(0.0000275).add(-0.2)
  d = d.addBands(d_scale, overwrite=True)
  d1 = d.clip(aoi)

  # export to asset
  fname = ee.String(image.get('system:index')).getInfo()
  export = ee.batch.Export.image.toAsset(
      image = d1.toFloat(),
      description= 'L9OL_' + fname,                         # 'L5TM' 'L7ET_' 'L8OL_' 'L9OL_'
      assetId = 'users/xxx/Landsat_oli/' +'L9OL_' + fname,  # Manually create image collection in GEE asset first 'Landsat_tm' 'Landsat_etm' 'Landsat_oli'
      region = aoi,
      scale = 30)
  export.start()
  print('exporting ' +fname + '--->done')


In [ ]:
aoi = ee.Geometry.Polygon([[[113.810, 22.580],[113.810, 22.140],[114.460, 22.140],[114.460, 22.580]]])
geom = aoi
df_metadata = pd.DataFrame()

#('LANDSAT/LT04/C02/T1_L2') LANDSAT/LT05/C02/T1_L2
#("LANDSAT/LE07/C02/T1_L2") LANDSAT/LC08/C02/T1_L2 LANDSAT/LC09/C02/T1_L2

dataset = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')   # change ImageCollection according to above
  .filterBounds(aoi)
  .filterDate('1970-01-01', '2022-12-31')
  .filter(ee.Filter.lte('CLOUD_COVER_LAND', 20)))

col_length = dataset.size().getInfo()
print(col_length)
if col_length > 0:
  col_list = dataset.toList(col_length)
  for i in range(0,col_length):
    print(i)
    img = ee.Image(col_list.get(i))
    p = img.getInfo()['properties']
    df_metadata1 = pd.DataFrame({'index': [p['system:index']], 'GEE_ID': [img.getInfo()['id']], 'LANDSAT_PRODUCT_ID': [p['LANDSAT_PRODUCT_ID']],
                               'SPACECRAFT_ID': [p['SPACECRAFT_ID']], 'SENSOR_ID': [p['SENSOR_ID']],
                               'Date': [p['DATE_ACQUIRED']], 'Time':[p['SCENE_CENTER_TIME']],
                               'WRS_TYPE':[p['WRS_TYPE']], 'WRS_PATH':[p['WRS_PATH']], 'WRS_ROW':[p['WRS_ROW']],
                               'CloudCover':[p['CLOUD_COVER']], 'SunAzimuth': [p['SUN_AZIMUTH']], 'SunElevation': [p['SUN_ELEVATION']], 'SunZenith': [90-p['SUN_ELEVATION']]})
    df_metadata = pd.concat([df_metadata, df_metadata1])
    preprocess(img)

df_metadata.to_csv('df_metadata.csv')
from google.colab import files
files.download('df_metadata.csv')

# Step 6 - Export from GEE to Google Drive and Download to PC


In [ ]:
# batch export image to Google Drive

imgcol = ee.ImageCollection("users/xxx/Landsat_oli")  # .filterDate('2009-10-16', '2012-12-31')    # change ImageCollection according to processed image
imgcol_length = imgcol.size().getInfo()
imgcol_list = imgcol.toList(imgcol_length)
for i in range(0,imgcol_length):
    print(i)
    img = ee.Image(imgcol_list.get(i))
    imgid = ee.Image.id(img).getInfo()
    print(imgid)
    task = ee.batch.Export.image.toDrive(**{'image': img, 'description': imgid, 'folder':'Colab Notebooks/export', 'scale': 30})
    task.start()


In [ ]:
# batch download from Google Drive
# run this part in local Python setup

import os
import shutil
import time
import ctypes

try:
    while True:
        size = 0
        Folderpath = 'G:/My Drive/Colab Notebooks export'
        for path, dirs, files in os.walk(Folderpath):
            for f in files:
                fp = os.path.join(path, f)
                size += os.path.getsize(fp)
        print(size)
        if size > 1500000000:
            file_names = os.listdir(Folderpath)
            for file_name in file_names:
                shutil.move(os.path.join(Folderpath, file_name), 'D:/landsat')
            ctypes.windll.user32.MessageBoxW(0, "Clear Google Drive", "Clear Google Drive", 1)
        time.sleep(60)

except KeyboardInterrupt:
    print('End')

In [ ]:
# batch delete asset (image collection) in GEE
# run this after download to clear GEE storage

delete_imgcol = "users/xxx/Landsat_oli"
s2_delete = ee.ImageCollection(delete_imgcol) # .filterDate('2015-01-01', '2022-09-30')
delete_length = s2_delete.size().getInfo()
for i in range(0,delete_length):
    print(i)
    del_list = s2_delete.toList(delete_length)
    delete_id = ee.Image.id(del_list.get(0)).getInfo()
    print(delete_id)
    ee.data.deleteAsset(delete_imgcol + "/" + delete_id)